In [1]:
import logging
import pickle

from pathlib import Path
from random import choice

from geoloc_imc_2023.cbg import CBG, get_prefix_from_ip
from geoloc_imc_2023.default import (
    ANCHOR_PATH
)

logging.basicConfig(level=logging.INFO)

NB_TARGET = 10
NB_VP = 10

ripe_credentials = {
    "username": "timur.friedman@sorbonne-universite.fr",
    "key": "b3d3d4fc-724e-4505-befe-1ad16a70dc87",
}

# load hitlist
hitlist_file = Path(".") / "../datasets/parsed_hitlist.pickle"
with open(hitlist_file, "rb") as f:
    hitlist_addr = pickle.load(f)

# load anchors
anchor_file = "../datasets/anchors.pickle"
with open(anchor_file, "rb") as f:
    anchors = pickle.load(f)

targets= [choice(list(anchors)) for _ in range(0, NB_TARGET)]
vps = list(set(anchors).difference(set(targets)))[:NB_VP]

logging.info(f"nb targets: {len(targets)} : {[target for target in targets]}")
logging.info(f"nb_vps : {len(vps)} {[vp for vp in vps]}")

cbg = CBG(targets, vps, ripe_credentials, hitlist_addr, anchors)

# get target prefixes
target_prefixes = []
for target_addr in targets:
    target_prefix = get_prefix_from_ip(target_addr)
    target_prefixes.append(target_prefix)

# initialization: start measurements from all vps to targets
init_measurement_ids = cbg.start_init_measurements(target_prefixes)

# retreive measurement results from RIPE Atlas
logging.info(init_measurement_ids)

INFO:root:nb targets: 10 : ['37.10.41.14', '103.196.37.98', '178.251.24.49', '207.246.74.246', '103.122.190.141', '192.42.42.166', '152.195.206.35', '37.252.251.26', '188.172.218.22', '63.222.190.5']
INFO:root:nb_vps : 10 ['34.97.153.243', '212.9.170.190', '107.162.219.5', '45.135.151.45', '176.74.17.75', '200.61.6.78', '212.72.227.133', '216.37.47.154', '68.64.60.178', '94.203.76.219']
INFO:root:starting measurement for target_prefix='37.10.41.0' with ['37.10.41.1', '37.10.41.2', '37.10.41.3'] with 10 self.anchors
INFO:root:starting measurement for target_prefix='103.196.37.0' with ['103.196.37.1', '103.196.37.2', '103.196.37.10'] with 10 self.anchors
INFO:root:starting measurement for target_prefix='178.251.24.0' with ['178.251.24.1', '178.251.24.2', '178.251.24.3'] with 10 self.anchors
INFO:root:starting measurement for target_prefix='207.246.74.0' with ['207.246.74.4', '207.246.74.7', '207.246.74.9'] with 10 self.anchors
INFO:root:starting measurement for target_prefix='103.122.190

In [2]:
import pickle

# save results
out_file = f"../results/init_measurement_ids.pickle"
print(out_file)
with open(out_file, "wb") as f:
    pickle.dump(init_measurement_ids,f)

../results/init_measurement_ids.pickle


## get init results

In [1]:
import pickle
from pathlib import Path

from geoloc_imc_2023.query_api import retreive_single_measurement

measurement_ids_file = Path(".") / "../results/init_measurement_ids.pickle"
with open(measurement_ids_file, "rb") as f:
    init_measurement_ids = pickle.load(f)

# load anchors
anchor_file = "../datasets/anchors.pickle"
with open(anchor_file, "rb") as f:
    anchors = pickle.load(f)

ripe_credentials = {
    "username": "timur.friedman@sorbonne-universite.fr",
    "key": "b3d3d4fc-724e-4505-befe-1ad16a70dc87",
}

measurement_results = {}
for prefix in init_measurement_ids:
    for measurement_id in init_measurement_ids[prefix]:
        measurement_results[prefix] = retreive_single_measurement(measurement_id, anchors, ripe_credentials["key"])

In [2]:
import pickle

# save results
out_file = f"../results/init_measurement.pickle"
print(out_file)
with open(out_file, "wb") as f:
    pickle.dump(measurement_results,f)

../results/init_measurement.pickle


## SECOND PART: get ip geoloc

In [1]:
import logging
import pickle

from pathlib import Path
from random import choice

from geoloc_imc_2023.cbg import CBG, get_prefix_from_ip
from geoloc_imc_2023.default import (
    ANCHOR_PATH
)

logging.basicConfig(level=logging.INFO)

NB_TARGET = 10
NB_VP = 10

ripe_credentials = {
    "username": "timur.friedman@sorbonne-universite.fr",
    "key": "b3d3d4fc-724e-4505-befe-1ad16a70dc87",
}

# load hitlist
hitlist_file = Path(".") / "../datasets/parsed_hitlist.pickle"
with open(hitlist_file, "rb") as f:
    hitlist_addr = pickle.load(f)

# load anchors
anchor_file = "../datasets/anchors.pickle"
with open(anchor_file, "rb") as f:
    anchors = pickle.load(f)

targets= [choice(list(anchors)) for _ in range(0, NB_TARGET)]
vps = list(set(anchors).difference(set(targets)))[:NB_VP]

logging.info(f"nb targets: {len(targets)} : {[target for target in targets]}")
logging.info(f"nb_vps : {len(vps)} {[vp for vp in vps]}")

cbg = CBG(targets, vps, ripe_credentials, hitlist_addr, anchors)

measurement_ids = cbg.start_target_measurements(targets, vps)

INFO:root:nb targets: 10 : ['134.169.3.165', '85.197.82.113', '81.211.204.45', '185.244.192.10', '212.29.44.199', '103.2.186.195', '23.157.112.130', '124.157.73.186', '131.100.0.158', '185.5.201.10']
INFO:root:nb_vps : 10 ['46.17.200.235', '81.169.160.78', '51.89.117.81', '137.193.69.236', '194.1.149.210', '77.220.233.1', '182.161.73.41', '185.138.55.234', '194.50.92.201', '92.38.178.149']


In [2]:
import pickle

# save results
out_file = f"../results/target_measurement_ids.pickle"
print(out_file)
with open(out_file, "wb") as f:
    pickle.dump(measurement_ids,f)

../results/target_measurement_ids.pickle


In [14]:
import pickle
from pathlib import Path

from geoloc_imc_2023.query_api import retreive_single_measurement

measurement_ids_file = Path(".") / "../results/target_measurement_ids.pickle"
with open(measurement_ids_file, "rb") as f:
    target_measurement_ids = pickle.load(f)

# load anchors
anchor_file = "../datasets/anchors.pickle"
with open(anchor_file, "rb") as f:
    anchors = pickle.load(f)

ripe_credentials = {
    "username": "timur.friedman@sorbonne-universite.fr",
    "key": "b3d3d4fc-724e-4505-befe-1ad16a70dc87",
}

measurement_results = []
for target_addr in target_measurement_ids:
    measurement_id = target_measurement_ids[target_addr]
    measurement_results.append(retreive_single_measurement(measurement_id, anchors, ripe_credentials["key"]))

In [15]:
import pickle

# save results
out_file = f"../results/target_measurement.pickle"
print(out_file)
with open(out_file, "wb") as f:
    pickle.dump(measurement_results,f)

../results/target_measurement.pickle


## Apply CBG methodology

In [1]:
from pathlib import Path
import random
import pickle
from copy import copy

from geoloc_imc_2023.helpers import polygon_centroid, haversine, circle_intersections
from geoloc_imc_2023.draw_results import draw_results

def select_best_guess_centroid(target_addr, measurement_results):
    """
    Find the best guess
    that is the location of the vantage point closest to the centroid.
    """
    probe_circles = {}
    print(f"NUMBER OF MEASUREMENT RESULTS FOR TARGET IP: {target_addr} : {len(measurement_results)}")
    for _, result in enumerate(measurement_results):
        vp_ip = result["node"]
        lat = result["vp_lat"]
        lon = result["vp_lon"]
        min_rtt = result["min_rtt"]

        # too inflated RTT means that measurement will not provide usefull info
        if isinstance(min_rtt, float):
                probe_circles[vp_ip] = (
                    lat,
                    lon,
                    min_rtt,
                    None,
                    None,
                )
                print(f"vp_anchor = {vp_ip} with results: {min_rtt}")
    print()

    # draw circles
    if not probe_circles: return

    intersections = circle_intersections(probe_circles.values(), speed_threshold=4/9)
    print("intersecion:", intersections)
    target = (anchors[target_addr]["latitude"], anchors[target_addr]["longitude"])

    centroid = None
    if intersections:
        centroid = polygon_centroid(intersections)
        print("centroid:", centroid)
        target_description = anchors[target_addr]

        if centroid:
            for ip_src, circle in probe_circles.items():

                measured_d = haversine((circle[0], circle[1]), centroid)
                calculated_d = haversine((target_description["latitude"], target_description["longitude"]), centroid)

                print("ip_src:",ip_src,"dst (measured):", measured_d, "dst (calculated):", calculated_d)
    
    draw_results(copy(probe_circles), intersections,centroid, target)

    return centroid

in_file = Path(".") / "../results/target_measurement.pickle"
with open(in_file, "rb") as f:
    measurements = pickle.load(f)

anchor_file = "../datasets/anchors.pickle"
with open(anchor_file, "rb") as f:
    anchors = pickle.load(f)

geoloc_results = {}
for i, results in enumerate(measurements):

    for target_addr in results:

        print("available measurements for ip:",len(results[target_addr]))
        shortest_delay_results = results[target_addr]
        print(shortest_delay_results)

        centroid = select_best_guess_centroid(target_addr, shortest_delay_results)
        if centroid:
            geoloc_results[target_addr] = {
                "estimated_lat": centroid[0],
                "estimated_lon": centroid[1],
            }

geoloc_out_file = Path(".") / "../results/all_anchors_towards_all_anchors_geoloc.pickle"
with open(geoloc_out_file, "wb") as f:
    pickle.dump(geoloc_results, f)


available measurements for ip: 10
[{'node': '77.220.233.1', 'min_rtt': 8.608871, 'rtt_list': [8.64558, 8.608871, 8.610858], 'vp_lat': 51.0395, 'vp_lon': 13.7285}, {'node': '185.138.55.234', 'min_rtt': 11.383693, 'rtt_list': [11.396568, 11.399771, 11.383693], 'vp_lat': 51.1905, 'vp_lon': 6.8675}, {'node': '194.50.92.201', 'min_rtt': 13.820218, 'rtt_list': [13.9291, 13.969491, 13.820218], 'vp_lat': 50.1375, 'vp_lon': 8.7375}, {'node': '81.169.160.78', 'min_rtt': 17.974235, 'rtt_list': [18.085142, 17.974235, 17.991283], 'vp_lat': 52.5175, 'vp_lon': 13.3185}, {'node': '51.89.117.81', 'min_rtt': 21.15478, 'rtt_list': [21.401514, 21.15478, 21.212063], 'vp_lat': 50.3995, 'vp_lon': 8.0775}, {'node': '137.193.69.236', 'min_rtt': 21.861116, 'rtt_list': [21.928351, 21.918153, 21.861116], 'vp_lat': 48.0785, 'vp_lon': 11.6505}, {'node': '46.17.200.235', 'min_rtt': 47.372811, 'rtt_list': [47.549715, 47.380912, 47.372811], 'vp_lat': 55.7275, 'vp_lon': 37.6695}, {'node': '194.1.149.210', 'min_rtt': 56

available measurements for ip: 10
[{'node': '194.50.92.201', 'min_rtt': 0.689657, 'rtt_list': [0.859012, 0.689657, 0.867083], 'vp_lat': 50.1375, 'vp_lon': 8.7375}, {'node': '51.89.117.81', 'min_rtt': 1.863281, 'rtt_list': [1.89205, 1.863281, 1.879952], 'vp_lat': 50.3995, 'vp_lon': 8.0775}, {'node': '185.138.55.234', 'min_rtt': 5.064167, 'rtt_list': [5.088473, 5.064167, 5.299531], 'vp_lat': 51.1905, 'vp_lon': 6.8675}, {'node': '137.193.69.236', 'min_rtt': 9.632505, 'rtt_list': [9.898937, 9.632505, 9.803152], 'vp_lat': 48.0785, 'vp_lon': 11.6505}, {'node': '81.169.160.78', 'min_rtt': 10.429009, 'rtt_list': [10.585796, 10.450765, 10.429009], 'vp_lat': 52.5175, 'vp_lon': 13.3185}, {'node': '77.220.233.1', 'min_rtt': 11.730159, 'rtt_list': [11.737109, 11.7377, 11.730159], 'vp_lat': 51.0395, 'vp_lon': 13.7285}, {'node': '46.17.200.235', 'min_rtt': 41.252394, 'rtt_list': [41.369996, 41.321695, 41.252394], 'vp_lat': 55.7275, 'vp_lon': 37.6695}, {'node': '194.1.149.210', 'min_rtt': 46.488751, '

available measurements for ip: 10
[{'node': '46.17.200.235', 'min_rtt': 60.851871, 'rtt_list': [62.073395, 60.851871, 61.011974], 'vp_lat': 55.7275, 'vp_lon': 37.6695}, {'node': '185.138.55.234', 'min_rtt': 103.093763, 'rtt_list': [103.351694, 103.14916, 103.093763], 'vp_lat': 51.1905, 'vp_lon': 6.8675}, {'node': '51.89.117.81', 'min_rtt': 107.221524, 'rtt_list': [107.485857, 107.359224, 107.221524], 'vp_lat': 50.3995, 'vp_lon': 8.0775}, {'node': '137.193.69.236', 'min_rtt': 109.877044, 'rtt_list': [110.258176, 109.877044, 110.199861], 'vp_lat': 48.0785, 'vp_lon': 11.6505}, {'node': '194.50.92.201', 'min_rtt': 114.027642, 'rtt_list': [114.027642, 114.369839, 114.427317], 'vp_lat': 50.1375, 'vp_lon': 8.7375}, {'node': '81.169.160.78', 'min_rtt': 117.26789, 'rtt_list': [117.447715, 117.374488, 117.26789], 'vp_lat': 52.5175, 'vp_lon': 13.3185}, {'node': '194.1.149.210', 'min_rtt': 147.013738, 'rtt_list': [147.304281, 147.17831, 147.013738], 'vp_lat': 41.3195, 'vp_lon': 19.8215}, {'node': 

available measurements for ip: 10
[{'node': '194.50.92.201', 'min_rtt': 3.693595, 'rtt_list': [4.130199, 3.797668, 3.693595], 'vp_lat': 50.1375, 'vp_lon': 8.7375}, {'node': '51.89.117.81', 'min_rtt': 4.888966, 'rtt_list': [5.013693, 4.888966, 4.906326], 'vp_lat': 50.3995, 'vp_lon': 8.0775}, {'node': '185.138.55.234', 'min_rtt': 7.713955, 'rtt_list': [7.835603, 9.388752, 7.713955], 'vp_lat': 51.1905, 'vp_lon': 6.8675}, {'node': '137.193.69.236', 'min_rtt': 13.15879, 'rtt_list': [13.15879, 13.38121, 13.255654], 'vp_lat': 48.0785, 'vp_lon': 11.6505}, {'node': '77.220.233.1', 'min_rtt': 13.348666, 'rtt_list': [13.348666, 15.251535, 14.578976], 'vp_lat': 51.0395, 'vp_lon': 13.7285}, {'node': '81.169.160.78', 'min_rtt': 17.543958, 'rtt_list': [17.760462, 17.599671, 17.543958], 'vp_lat': 52.5175, 'vp_lon': 13.3185}, {'node': '46.17.200.235', 'min_rtt': 39.47006, 'rtt_list': [39.50976, 39.512361, 39.47006], 'vp_lat': 55.7275, 'vp_lon': 37.6695}, {'node': '194.1.149.210', 'min_rtt': 49.069864, 

available measurements for ip: 10
[{'node': '185.138.55.234', 'min_rtt': 3.764978, 'rtt_list': [3.868713, 4.017806, 3.764978], 'vp_lat': 51.1905, 'vp_lon': 6.8675}, {'node': '194.50.92.201', 'min_rtt': 7.022847, 'rtt_list': [43.354141, 7.022847, 7.376084], 'vp_lat': 50.1375, 'vp_lon': 8.7375}, {'node': '51.89.117.81', 'min_rtt': 9.223237, 'rtt_list': [9.223237, 9.322556, 9.269921], 'vp_lat': 50.3995, 'vp_lon': 8.0775}, {'node': '77.220.233.1', 'min_rtt': 13.417591, 'rtt_list': [13.417591, 13.565385, 13.495424], 'vp_lat': 51.0395, 'vp_lon': 13.7285}, {'node': '137.193.69.236', 'min_rtt': 16.434264, 'rtt_list': [16.434264, 55.384366, 16.566031], 'vp_lat': 48.0785, 'vp_lon': 11.6505}, {'node': '81.169.160.78', 'min_rtt': 19.083498, 'rtt_list': [19.114385, 19.112195, 19.083498], 'vp_lat': 52.5175, 'vp_lon': 13.3185}, {'node': '194.1.149.210', 'min_rtt': 49.153198, 'rtt_list': [57.728916, 49.153198, 55.503608], 'vp_lat': 41.3195, 'vp_lon': 19.8215}, {'node': '46.17.200.235', 'min_rtt': 49.7

available measurements for ip: 10
[{'node': '92.38.178.149', 'min_rtt': 201.070229, 'rtt_list': [201.070229, 216.374623, 201.350853], 'vp_lat': 35.6475, 'vp_lon': 139.7885}, {'node': '46.17.200.235', 'min_rtt': 274.538682, 'rtt_list': [274.550982, 274.636084, 274.538682], 'vp_lat': 55.7275, 'vp_lon': 37.6695}, {'node': '182.161.73.41', 'min_rtt': 275.914755, 'rtt_list': [275.927096, 275.94861, 275.914755], 'vp_lat': 1.4285, 'vp_lon': 103.7975}, {'node': '77.220.233.1', 'min_rtt': 295.349937, 'rtt_list': [301.83656, 295.349937, 296.086459], 'vp_lat': 51.0395, 'vp_lon': 13.7285}, {'node': '81.169.160.78', 'min_rtt': 300.475975, 'rtt_list': [300.629813, 300.475975, 313.657714], 'vp_lat': 52.5175, 'vp_lon': 13.3185}, {'node': '185.138.55.234', 'min_rtt': 307.699984, 'rtt_list': [317.036974, 307.838126, 307.699984], 'vp_lat': 51.1905, 'vp_lon': 6.8675}, {'node': '194.50.92.201', 'min_rtt': 312.478105, 'rtt_list': [312.478105, 315.911304, 317.477628], 'vp_lat': 50.1375, 'vp_lon': 8.7375}, {'

available measurements for ip: 10
[{'node': '92.38.178.149', 'min_rtt': 114.943457, 'rtt_list': [115.461427, 115.376057, 114.943457], 'vp_lat': 35.6475, 'vp_lon': 139.7885}, {'node': '185.138.55.234', 'min_rtt': 147.622119, 'rtt_list': [147.78565, 147.718641, 147.622119], 'vp_lat': 51.1905, 'vp_lon': 6.8675}, {'node': '194.50.92.201', 'min_rtt': 150.254263, 'rtt_list': [150.948133, 150.820587, 150.254263], 'vp_lat': 50.1375, 'vp_lon': 8.7375}, {'node': '81.169.160.78', 'min_rtt': 154.843257, 'rtt_list': [155.405595, 154.843257, 155.142922], 'vp_lat': 52.5175, 'vp_lon': 13.3185}, {'node': '51.89.117.81', 'min_rtt': 155.057368, 'rtt_list': [155.257865, 155.181305, 155.057368], 'vp_lat': 50.3995, 'vp_lon': 8.0775}, {'node': '77.220.233.1', 'min_rtt': 155.862695, 'rtt_list': [156.248116, 155.862695, 155.990259], 'vp_lat': 51.0395, 'vp_lon': 13.7285}, {'node': '137.193.69.236', 'min_rtt': 160.166423, 'rtt_list': [160.529618, 160.601312, 160.166423], 'vp_lat': 48.0785, 'vp_lon': 11.6505}, {'

available measurements for ip: 10
[{'node': '182.161.73.41', 'min_rtt': 127.934911, 'rtt_list': [127.994693, 127.962293, 127.934911], 'vp_lat': 1.4285, 'vp_lon': 103.7975}, {'node': '92.38.178.149', 'min_rtt': 146.755541, 'rtt_list': [150.578632, 146.755541, 146.779731], 'vp_lat': 35.6475, 'vp_lon': 139.7885}, {'node': '194.50.92.201', 'min_rtt': 289.476458, 'rtt_list': [289.5891, 295.42968, 289.476458], 'vp_lat': 50.1375, 'vp_lon': 8.7375}, {'node': '185.138.55.234', 'min_rtt': 292.429439, 'rtt_list': [292.475302, 292.708933, 292.429439], 'vp_lat': 51.1905, 'vp_lon': 6.8675}, {'node': '51.89.117.81', 'min_rtt': 293.28594, 'rtt_list': [293.441271, 293.397775, 293.28594], 'vp_lat': 50.3995, 'vp_lon': 8.0775}, {'node': '77.220.233.1', 'min_rtt': 297.26943, 'rtt_list': [297.26943, 297.400337, 297.511009], 'vp_lat': 51.0395, 'vp_lon': 13.7285}, {'node': '81.169.160.78', 'min_rtt': 300.249751, 'rtt_list': [300.344845, 300.267626, 300.249751], 'vp_lat': 52.5175, 'vp_lon': 13.3185}, {'node': 

available measurements for ip: 10
[{'node': '51.89.117.81', 'min_rtt': 139.421137, 'rtt_list': [139.425857, 139.421137, 139.506562], 'vp_lat': 50.3995, 'vp_lon': 8.0775}, {'node': '77.220.233.1', 'min_rtt': 141.368004, 'rtt_list': [141.662129, 142.327789, 141.368004], 'vp_lat': 51.0395, 'vp_lon': 13.7285}, {'node': '194.50.92.201', 'min_rtt': 143.56933, 'rtt_list': [143.56933, 145.121881, 143.752553], 'vp_lat': 50.1375, 'vp_lon': 8.7375}, {'node': '185.138.55.234', 'min_rtt': 144.459195, 'rtt_list': [144.533075, 144.473619, 144.459195], 'vp_lat': 51.1905, 'vp_lon': 6.8675}, {'node': '92.38.178.149', 'min_rtt': 152.870303, 'rtt_list': [152.881614, 152.870303, 153.504966], 'vp_lat': 35.6475, 'vp_lon': 139.7885}, {'node': '81.169.160.78', 'min_rtt': 154.12173, 'rtt_list': [154.232997, 154.12173, 154.132955], 'vp_lat': 52.5175, 'vp_lon': 13.3185}, {'node': '46.17.200.235', 'min_rtt': 165.651287, 'rtt_list': [165.920693, 165.651287, 165.752589], 'vp_lat': 55.7275, 'vp_lon': 37.6695}, {'node

available measurements for ip: 10
[{'node': '194.50.92.201', 'min_rtt': 30.771187, 'rtt_list': [32.399636, 30.771187, 35.149623], 'vp_lat': 50.1375, 'vp_lon': 8.7375}, {'node': '51.89.117.81', 'min_rtt': 33.965698, 'rtt_list': [33.965698, 34.010588, 34.042166], 'vp_lat': 50.3995, 'vp_lon': 8.0775}, {'node': '194.1.149.210', 'min_rtt': 34.578212, 'rtt_list': [39.481473, 34.578212, 35.056427], 'vp_lat': 41.3195, 'vp_lon': 19.8215}, {'node': '77.220.233.1', 'min_rtt': 36.319715, 'rtt_list': [36.751281, 36.319715, 36.431], 'vp_lat': 51.0395, 'vp_lon': 13.7285}, {'node': '81.169.160.78', 'min_rtt': 37.228402, 'rtt_list': [37.774084, 37.228402, 37.319711], 'vp_lat': 52.5175, 'vp_lon': 13.3185}, {'node': '137.193.69.236', 'min_rtt': 38.504276, 'rtt_list': [38.504276, 38.627956, 38.618183], 'vp_lat': 48.0785, 'vp_lon': 11.6505}, {'node': '185.138.55.234', 'min_rtt': 47.422365, 'rtt_list': [47.608155, 47.504143, 47.422365], 'vp_lat': 51.1905, 'vp_lon': 6.8675}, {'node': '46.17.200.235', 'min_rt

In [9]:
geoloc_out_file = Path(".") / "../results/all_anchors_towards_all_anchors_geoloc.pickle"
with open(geoloc_out_file, "rb") as f:
    geoloc_results = pickle.load(f)


In [11]:
from geoloc_imc_2023.helpers import distance

for i, target_addr in enumerate(geoloc_results):
    if i > 15: break
    target_lat = anchors[target_addr]["latitude"]
    target_lon = anchors[target_addr]["longitude"]

    estimated_lat = geoloc_results[target_addr]["estimated_lat"]
    estimated_lon = geoloc_results[target_addr]["estimated_lon"]

    d_error = distance(target_lat, estimated_lat, target_lon, estimated_lon)
    print(f"geoloc results for target ip: {target_addr} distance error: {d_error}")

geoloc results for target ip: 145.220.0.55 distance error: 3.981388122601496
geoloc results for target ip: 5.28.0.17 distance error: 166.404396745
geoloc results for target ip: 193.171.255.2 distance error: 0.8226714520441889
geoloc results for target ip: 192.65.184.54 distance error: 3.5015730277578623
geoloc results for target ip: 197.80.104.36 distance error: 19.16556336974885
geoloc results for target ip: 200.7.84.24 distance error: 26.886725742565762
geoloc results for target ip: 76.26.115.194 distance error: 85.76798202864943
geoloc results for target ip: 194.150.191.46 distance error: 2.5171887386945353
geoloc results for target ip: 199.10.66.231 distance error: 6.835654928240526
